# 从零开始部署领域大模型应用——以DST-GPT为例

![Bg Image](../assets/BG.png)

In [ ]:
# INSTALL PACKAGES
# using conda env | python=3.8.18

%pip install python-dotenv
%pip install langchain==0.1.4
%pip install langchain-openai==0.0.4

# %pip install langchain-community==0.0.16
# %pip install langchain-core==0.1.16
# %pip install langsmith==0.0.87

# [Quickstart | 🦜️🔗 Langchain](https://python.langchain.com/docs/get_started/quickstart)

> 由于我们使用的是OpenAI的服务，使用此应用时可能需要用到魔法  
> 但如果你的OpenAI API key是中转类型的，可能不需要

## 1. 配置OpenAI API key

In [ ]:
# Data Privacy
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

# Set up your API key from OpenAI
api_key = os.getenv('OPENAI_API_KEY')
base_url=os.getenv('OPENAI_BASE_URL') 

# Print
print(api_key)
print(base_url)

## 2. 创建一个基本LLM应用

然后，我们可以初始化模型：

In [ ]:
# Initialize the model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=api_key, # alias:api_key
    base_url="", # If you use proxy api key, set base_url as needed 
    model="gpt-3.5-turbo-0125", 
    temperature=0.7, #default
    )

一旦您安装并初始化了您选择的LLM，我们就可以尝试使用它了！让我们问它 '你调用的是哪个基模型'。
> 大模型具有随机性的特点，完全相同的输入可以得到不同（但相近）的回答

In [ ]:
for i in range(5):
    response = llm.invoke("你用的是哪个底座模型")
    print(f"Response {i+1}: {response.content}")

## 3. 使用向量数据库外挂知识增强LLM在特定领域的能力——以Don't Starve Together(DST)为例

![Sup1](../assets/DST1.png)

我们向gpt-3.5-turbo-0125提一个关于DST这款游戏的问题，即“Don't Starve Together这款游戏截止2023年12月，在Steam平台上共有多少评论？”。由于gpt-3.5-turbo-0125的训练语料数据仅截止到2021年过，显然其目前无法回答这个问题。此外，此问题过于领域化，基座大模型训练语料基本不可能包含相关信息，我们这里通过向量数据库外挂知识的方式，增加LLM在特地领域的问答能力。

In [ ]:
response=llm.invoke("Don't Starve Together这款游戏截止2023年12月，在Steam平台上共有多少评论？")
print(response.content)

上述回答为大模型幻觉（Illusion）

为了正确回答此问题，我们需要为LLM添加额外知识。我们可以通过`Retrieval`来做到这一点。当您有太多数据无法直接传递时，`Retrieval`很有用。然后，您可以使用`Retriever`仅获取最相关的片段并将其传递。
在这个过程中，我们将从 `Retriever` 中查找相关文档，然后将它们传递到`Prompt`中。在本例中，我们将填充一个`vectorstore`并将其用作`Retriever`。有关 `vectorstore` 的更多信息，请参阅[此文档](https://python.langchain.com/docs/modules/data_connection/vectorstores)。

首先，我们需要加载要索引的数据。这里我们的数据来源于网页信息，为此，我们将使用 WebBaseLoader。这需要安装 BeautifulSoup：

In [ ]:
%pip install beautifulsoup4

之后，我们可以导入并使用 WebBaseLoader。

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
dst_wiki_mainpage_url="https://dontstarve.fandom.com/wiki/Don%27t_Starve_Together#Return_of_Them"
loader = WebBaseLoader(dst_wiki_mainpage_url)

docs = loader.load()



In [ ]:
print(docs[0].page_content[100:180])

清洗数据，去掉"\n"和"\t"

In [ ]:
# Get the text of the first Document instance
original_text = docs[0].page_content

# Use the replace method to remove newline and tab characters
cleaned_text = original_text.replace("\n", "").replace("\t", "")

# Update the text attribute of the Document instance
docs[0].page_content = cleaned_text

# Print the length of the cleaned text and the cleaned text itself for confirmation
print("文档字符串长度为：", len(docs[0].page_content))
#print(docs[0].page_content)

截取这个问题的回答相关信息源，打印。

In [ ]:
content = docs[0].page_content
start_index = content.find("As of 2023-12-05 on Steam, DST had")
end_index = content.find("new development takes place almost exclusively in DST.")
relevant_content = content[start_index:end_index]
print(relevant_content)

接下来，我们需要将其索引到`vectorstore`中。这需要一些组件，即`embedding`和`vectorstore`。
对于`embedding`，我们再次使用OpenAI的示例。

确保您安装了“langchain_openai”包并设置了适当的环境变量（这些变量与 所需的环境变量相同LLM）。

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model= "text-embedding-ada-002",#default
    openai_api_key=api_key, # alias:api_key
    base_url=base_url, # If you use proxy api key, set base_url as needed 
    chunk_size=1000,#default
)

现在，我们可以使用这个`embedding`将文档摄取到`vectorstore`中。为了简单起见，我们将使用一个简单的本地`vectorstore`: `Chroma`

首先，我们需要为此安装所需的软件包：

In [ ]:
%pip install chromadb

然后我们可以构建我们的索引：

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

vector = Chroma.from_documents(
    documents=docs, 
    embedding=embeddings,
    )

In [ ]:
print(len(vector.get()['ids']))
print(type(vector))


现在，我们已经在`vectorstore`中索引了这些数据，我们将创建一个`retrieval chain`。该`chain`将接受传入的问题，查找相关文档，然后将这些文档与原始问题一起传递到一个LLM中，并要求它回答原始问题。

首先，让我们设置一个`chain`，它接受一个问题和检索到的文档并生成一个答案。

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(
    llm=llm, 
    prompt=prompt,
    )

我们现在可以调用这个chain了。

In [ ]:
from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "DST这款游戏截止2023年12月，在Steam平台上共有多少评论？"})
print(response["answer"])

上述回答完全正确 参照[Wiki Fandom](https://dontstarve.fandom.com/wiki/Don%27t_Starve_Together#Return_of_Them)  
![DST2.png](../assets/DST2.png)

## 4. 使用异步调用llm

参考[langchain官方文档](https://python.langchain.com/docs)

In [ ]:
response=await llm.ainvoke("你用的什么基模型")
print(response.content)
print(type(response.content))